#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 2:  Link Analysis -- PageRank + SEO

### 100 points [5% of your final grade]

### Due: Tuesday, February 21, 2017 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test PageRank over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 24, 2017 at 11:59pm.

# Part 1: PageRank (70 points)

## A Twitter-Mentioning Graph

In this assignment, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their mentions of other Twitter users (so user = node, mention of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is mentioned by other users is more "impactful". 

Here is a toy example. Suppose you are given the following four tweets:

* **userID**: diane, **text**: "@bob Howdy!"
* **userID**: charlie, **text**: "Welcome @bob and @alice!"
* **userID**: bob, **text**: "Hi @charlie and @alice!"
* **userID**: alice, **text**: "Howdy!"

There are four short tweets generated by four users. The @mentions between users form a directed graph with four nodes and five edges. E.g., the "diane" node has a directed edge to the "bob" node. Note that a retweet also contain the "@", so it should be counted as a mention as well.

You should build a graph by parsing the tweets in the file we provide called *pagerank.json*.

**Notes:**

* The edges are binary and directed. If Bob mentions Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user mentions herself, ignore it.
* Correctly parsing @mentions in a tweet is error-prone. Use the entities field.
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [1]:
# Here define your function for building the graph by parsing the input file of tweets
# Insert as many cells as you want
import json
import copy
import operator
from collections import defaultdict

in_links = defaultdict(set)
out_links = defaultdict(set)
page_rank = {}
nodes = 0
edges = 0
d = 0.9
def create_graph():
    global edges
    with open ('pagerank.json') as data_file:
        for line in data_file:
            data = json.loads(line)
            #print data["user"]["id"]
            source_id = data["user"]["id"]
            entities = data["entities"]["user_mentions"]
            for entity in entities:
                destination_id = entity["id"]
                if source_id == destination_id:
                    continue
                out_links[source_id].add(destination_id)
                if destination_id not in out_links:
                    out_links[destination_id] = set()
                in_links[destination_id].add(source_id)
                if source_id not in in_links:
                    in_links[source_id] = set()

def dump_graphInfo():
    global nodes
    global edges
    nodes = len(out_links)
    print "nodes : " + str(nodes)
    edges = 0
    for node in out_links.keys():
        edges += len(out_links[node])
    print "edges : " + str(edges)
        
        
        
        
        
        
        
        
        
        

In [2]:

# Call your function to print out the size of the graph, i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
# However, if you encounter any memory issues, we recommend you write the graph into a file, and load it later.
create_graph()
dump_graphInfo()


nodes : 16430
edges : 24256


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## PageRank Implementation

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never mentioned and does not mention anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions (Numpy 1.12.0; Scipy 0.18.1).
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

Termination Condition - Top k pages remaining same. I have taken k as 10. So, when I get the same set of 10 pages in an interation, I break the loop and assume that the page rank scores have converged.

In [3]:
# Here add your code to implement a function called PageRanker
# Insert as many cells as you want

def normalize():
    total_pr = sum(page_rank.itervalues())
    for key, value in page_rank.items():
        page_rank[key] = float(value)/total_pr

def is_converged(curr, prev):
    if prev == None:
        return False
    count = 10
    for i in range(count):
        if curr[i][0] != prev[i][0]:
            return False
    return True

def PageRanker():
    #Initialisation
    curr = []
    prev = None
    teleport_probability = (1.0 - d) / nodes
    for node in out_links.keys():
        page_rank[node] = 1.0 / nodes
    iterations = 0
    #for i in range(0, iterations):
    while True:
        iterations += 1
        temp_copy = copy.deepcopy(page_rank)
        for node in out_links.keys():
            page_rank[node] = 0.0
            for incoming_node in in_links[node]:
                l = len(out_links[incoming_node])
                pr_l = temp_copy[incoming_node]
                page_rank[node] += float(pr_l) / l
            page_rank[node] *= d
            page_rank[node] += (1 - d) / nodes
        curr = sorted(page_rank.items(), key = operator.itemgetter(1), reverse=True)
        flag = is_converged(curr, prev)
        if flag == True:
            break
        else:
            prev = curr
    print "# iterations : " + str(iterations)
    normalize()

def dumpTopK(k):
    foo = 0
    for w in sorted(page_rank, key=page_rank.get, reverse=True):
        if foo == k:
            break
        print w, page_rank[w]
        foo += 1
                

In [4]:
# Now let's call your function on the graph you've built. Output the results.
PageRanker()
dumpTopK(10)

# iterations : 6
158314798 0.00769759690546
181561712 0.00650401460636
209708391 0.0060455333375
72064417 0.00551054473328
105119490 0.00506513219541
14268057 0.00465675266018
379961664 0.00440363300339
391037985 0.00423667138363
153074065 0.00423333528046
313525912 0.00407549014064


# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **awcv9kjlh scwrlkjf4e** --- two terms, lower case, no quote. As of today (Feb 7, 2017), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

http://people.tamu.edu/~navgupta14/

What's your strategy? (2-4 paragraphs)

1. Created a people's page on tamu domain. This domain has high reputation and thus gets inherited in personal page too. This adds to the authority factors and helps in indexing.
2. Embedding this string "awcv9kjlh scwrlkjf4e" at various places in the content of the page. This adds to the content relevance for the purpose of indexing.
3. Creating another web page on github.io(https://navgupta14.github.io/) and wordpress(https://navgupta14.wordpress.com/) and proving inlinks to the main page. This is to ensure that the main page has enough inlinks so as to enhance its page rank value.
4. Used "awcv9kjlh scwrlkjf4e" as the hyperlink to http://people.tamu.edu/~navgupta14/ . Anchor texts are a great way to improve indexing.
5. Added links to Google Trends, Youtube Trends, and new trending topics so as to make the page somewhat relevant to an unkown user and also to boost indexing.